In [ ]:
## load train model

In [ ]:
## run sampling


In [ ]:
## gather simulation input, test following predict algorithms


In [2]:
## test Bsyesian optimization
from sklearn import metrics
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt
import catboost as cb
from bayes_opt import BayesianOptimization


In [ ]:
class optimizationClass():
    # 初始化的时候把数据集传进来
    def __init__(self,df):
        self.df = df 
        
     # 定义一个评估指标的函数评估模型好坏   
    def reg_calculate(self,true, prediction): 
        mse = metrics.mean_squared_error(true, prediction)
        rmse = np.sqrt(mse)
        mae = metrics.mean_absolute_error(true, prediction)
        mape = np.mean(np.abs((true - prediction) / true)) * 100
        r2 = metrics.r2_score(true, prediction)
        rmsle = np.sqrt(metrics.mean_squared_log_error(true, prediction))
        # print("mse: {}, rmse: {}, mae: {}, mape: {}, r2: {}, rmsle: {}".format(mse, rmse, mae, mape, r2, rmsle))
        # return mse, rmse, mae, mape, r2, rmsle
        return {"mse": mse, "rmse": rmse, "mae": mae, "mape": mape, "r2": r2, "rmsle": rmsle}    
    
    # 优化函数 参数为你需要调参的 hyper Parameter 这里用catBoost作为样例
    def optimization_function(self,iterations,learning_rate,depth,l2_leaf_reg):
        parameterDict = {"iterations":int(iterations),"learning_rate": float(learning_rate), "depth": int(depth),
                         "l2_leaf_reg":float(l2_leaf_reg),
                         "task_type":"CPU","logging_level":"Silent"}
        CB_Regressor=cb.CatBoostRegressor(**parameterDict)
        CB_Regressor.fit(self.df["X_train"], self.df["Y_train"])
        Y_pre=CB_Regressor.predict(self.df["X_test"])
        resDict =self.reg_calculate(self.df["Y_test"],Y_pre)
        return resDict["r2"]
    
    # 定义一下模型
    def run(self, init_points=2,n_iter=3):
        cb_bo = BayesianOptimization(
        self.optimization_function,
        {'iterations': (200, 5000),
        'learning_rate': (1e-6, 1e-2),
        'depth': (2, 15),
        'l2_leaf_reg': (0, 5)}
        )
        cb_bo.maximize(
                        init_points=init_points,
                        n_iter=n_iter)
        print("Final result:", cb_bo.max)
#     


In [ ]:
Opt = optimizationClass(df = {"X_train":X_Train, "X_test":X_Test, "Y_train":Y_Train, "Y_test":Y_Test})
Opt.run()

In [ ]:
from colmena.models import Result

result_list = []
result = Result(
    (('name',), {'a':1}),
    method="method",
    keep_inputs=True,
    task_info={"resources": {"num_cpus": 1, "num_gpus": 0}},
    resources={"num_cpus": 2, "num_gpus": 0, "num_threads": 2}
)
result_list.append(result)
result = Result(
    (('name',), {'a':1}),
    method="method",
    keep_inputs=True,
    task_info={"resources": {"num_cpus": 1, "num_gpus": 0}},
    resources={"num_cpus": 4, "num_gpus": 0, "num_threads": 2}
)
result_list.append(result)

features = [
    'resources,num_cpus',
    'resources,num_gpus',
    'resources,num_threads'
]
data = []
feature_values = {}
for result in result_list:
    for feature in features:
        feature = feature.split(',')
        for i in range(len(feature)):
            value = getattr(result, feature[i], None)
        feature_values[feature[-1]] = value
    data.append(feature_values)

print(data)

In [1]:
# preprocess data
from pathlib import Path
import os
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# path = Path('/home/lizz_lab/cse30019698/project/colmena/multisite_/finetuning-surrogates/analysis/runs')
# wsl local path 
path = Path('/home/lizz_lab/cse30019698/project/colmena/multisite_/finetuning-surrogates/analysis/runs/20240227_192900')
titles = ['sampling','simulation','training','inference']
def find_files_with_title(directory, title):
    result = {}
    base_level = str(directory).count(os.sep)
    for root, dirs, files in os.walk(directory):
        if root.count(os.sep) <= base_level+3:  # 只遍历三级子目录
            for file in files:
                if title in file:
                    relative_path = os.path.relpath(os.path.join(root, file), str(directory))
                    result[relative_path] = os.path.join(root, file)
    return result

features = [
    # "method",
    "message_sizes.inputs",
    # "worker_info.hostname",
    "resources.node_count",
    "resources.cpu_processes",
    "resources.cpu_threads",
    "time_running"]
        
def get_running_metrics(data_path,features):
    data = []
    for path in data_path:
        with open(path) as f:
            for line in f:
                log = json.loads(line)
                feature_values = {}
                for feature in features:
                    feature_value = log
                    for key in feature.split("."):
                        feature_value = feature_value.get(key)
                        if feature_value is None:
                            break
                    feature_values[feature] = feature_value
                data.append(feature_values)
    df = pd.DataFrame(data)
    return df


simulation_data = find_files_with_title(path, 'simulation')
print(simulation_data)
df = get_running_metrics(simulation_data.values(),features)
# label_encoder = LabelEncoder()
# encoded_labels = label_encoder.fit_transform(df['method'])
# df['method'] = encoded_labels


{}


In [3]:
print(df.head())
print(df.shape)

   message_sizes.inputs  resources.node_count resources.cpu_processes  \
0                  8733                     1                      16   
1                  7869                     1                      16   
2                  7869                     1                      16   
3                  1387                     1                      16   
4                  3549                     1                      16   

   resources.cpu_threads  time_running  
0                      1    450.060844  
1                      1    152.374982  
2                      1    191.483258  
3                      1      4.547907  
4                      1     33.308450  
(176, 5)


In [14]:
## test random forest
data = df.apply(pd.to_numeric, errors='coerce')
data = data.dropna()  # 删除缺失值
# 提取特征和目标变量
X = data.drop('time_running', axis=1)  # 特征
y = data['time_running']  # 目标变量

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 创建随机森林回归模型
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# 训练模型
rf.fit(X_train, y_train)

# 在测试集上进行预测
y_pred = rf.predict(X_test)

for i in range(len(y_pred)):
    print(f"预测值: {y_pred[i]}, 实际值: {y_test.iloc[i]}")


# 评估模型性能
mse = mean_squared_error(y_test, y_pred)
print(f"均方误差（MSE）: {mse}")

预测值: 68.22887702589544, 实际值: 64.45904402434826
预测值: 29.902914699492417, 实际值: 31.66200151713565
预测值: 207.94587903871346, 实际值: 218.76287353923544
预测值: 128.86375018477466, 实际值: 118.2591360528022
预测值: 71.78310965596246, 实际值: 75.06399099109694
预测值: 123.49271703878678, 实际值: 93.18448483897373
预测值: 2912.0254765669483, 实际值: 3033.612930558156
预测值: 109.40814889019356, 实际值: 130.4530840329826
预测值: 15.71897856842272, 实际值: 16.42160639585927
预测值: 444.73675419209235, 实际值: 531.904815223068
预测值: 333.1232807943023, 实际值: 315.3497639307752
预测值: 53.837410173350946, 实际值: 59.992288656998426
预测值: 634.306247070415, 实际值: 559.8373141619377
预测值: 287.53674668864693, 实际值: 323.5934388479218
预测值: 246.6296064982663, 实际值: 229.7090552840382
预测值: 75.78134263334873, 实际值: 88.68210051394999
预测值: 91.21208394491114, 实际值: 106.03658497519791
预测值: 102.85138550474866, 实际值: 111.69201503833756
预测值: 52.58134820472573, 实际值: 69.93717657169327
预测值: 371.294023298903, 实际值: 455.9879334741272
预测值: 158.39513198443657, 实际值: 162.2259537470527
预

In [13]:
# test xgboost
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

data = df.apply(pd.to_numeric, errors='coerce')
data = data.dropna()  # 删除缺失值
# 提取特征和目标变量
X = data.drop('time_running', axis=1)  # 特征
y = data['time_running']  # 目标变量

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 创建XGBoost回归模型
xgb_model = xgb.XGBRegressor(random_state=42)

# 训练模型
xgb_model.fit(X_train, y_train)

# 在测试集上进行预测
y_pred = xgb_model.predict(X_test)

# 逐行打印预测值和实际值进行比较
for i in range(len(y_pred)):
    print(f"预测值: {y_pred[i]}, 实际值: {y_test.iloc[i]}")

# 评估模型性能
mse = mean_squared_error(y_test, y_pred)
print(f"均方误差（MSE）: {mse}")

预测值: 66.5465087890625, 实际值: 64.45904402434826
预测值: 31.840682983398438, 实际值: 31.66200151713565
预测值: 206.184326171875, 实际值: 218.76287353923544
预测值: 139.84230041503906, 实际值: 118.2591360528022
预测值: 71.81390380859375, 实际值: 75.06399099109694
预测值: 129.35867309570312, 实际值: 93.18448483897373
预测值: 3017.46826171875, 实际值: 3033.612930558156
预测值: 113.20099639892578, 实际值: 130.4530840329826
预测值: 15.658439636230469, 实际值: 16.42160639585927
预测值: 457.42498779296875, 实际值: 531.904815223068
预测值: 323.9720153808594, 实际值: 315.3497639307752
预测值: 79.36309814453125, 实际值: 59.992288656998426
预测值: 565.822265625, 实际值: 559.8373141619377
预测值: 301.04541015625, 实际值: 323.5934388479218
预测值: 236.3675994873047, 实际值: 229.7090552840382
预测值: 80.52764892578125, 实际值: 88.68210051394999
预测值: 62.70760726928711, 实际值: 106.03658497519791
预测值: 97.02880859375, 实际值: 111.69201503833756
预测值: 63.95719528198242, 实际值: 69.93717657169327
预测值: 337.2482604980469, 实际值: 455.9879334741272
预测值: 167.89044189453125, 实际值: 162.2259537470527
预测值: 157.209045

In [13]:
import json

results = []
# 读取 JSON 文件
with open('/home/lizz_lab/cse30019698/project/colmena/multisite_/finetuning-surrogates/runs/20240227_012918/dft-md-24Feb26-213555-1434fe/simulation-results.json', 'r') as f:

    for line in f:
        data = json.loads(line)
        # print(data)
        # 修改每一行的 "cpu_processes"
        data['resources']['cpu_processes'] = '1'
        results.append(data)
        # 将修改后的数据写回 JSON 文件
with open('/home/lizz_lab/cse30019698/project/colmena/multisite_/finetuning-surrogates/runs/20240227_012918/simulation-results_1.json', 'w') as f:
    for data in results:
        json.dump(data, f)
        f.write('\n')